In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from env import host, username, password

## Acquire and Summarize Data

- Acquired data requirements:
  - join all tables
  - only properties with transaction in 2017
  - ensure no duplicate property ids >> use latest transaction date
  - properties must contain latitude and longitude

In [2]:
#Function to create database url.  Requires local env.py with host, username and password. 
# No function help text provided as we don't want the user to access it and display their password on the screen
def get_db_url(db_name,user=username,password=password,host=host):
    url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
    return url

#Function to get new data from Codeup server
def getNewZillowData():
    """
    Retrieves zillow dataset from Codeup DB and stores a local csv file
    Returns: Pandas dataframe
    """
    db_name= 'zillow'
    filename='zillow.csv'
    sql = """
    SELECT *
    FROM properties_2017
        JOIN predictions_2017 USING(parcelid)
        LEFT JOIN airconditioningtype USING(airconditioningtypeid)
        LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
        LEFT JOIN buildingclasstype USING(buildingclasstypeid)
        LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
        LEFT JOIN propertylandusetype USING (propertylandusetypeid)
        LEFT JOIN storytype USING (storytypeid)
        LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
    WHERE transactiondate LIKE '2017%%' 
        AND latitude IS NOT NULL
        AND longitude IS NOT NULL;
    """
    #Read SQL from file
    df = pd.read_sql(sql,get_db_url(db_name))
    #Drop ID columns
    df.drop(columns=['id'],inplace=True)
    #write to disk - writes index as col 0:
    df.to_csv(filename)
    return df

#Function to get data from local file or Codeup server 
def getZillowData():
    """
    Retrieves Zillow dataset from working directory or Codeup DB. Stores a local copy if one did not exist.
    Returns: Pandas dataframe of zillow data
    """
    #Set filename
    filename = 'zillow.csv'

    if os.path.isfile(filename): #check if file exists in WD
        #grab data, set first column as index
        return pd.read_csv(filename,index_col=[0])
    else: #Get data from SQL db
        df = getNewZillowData()
    return df

In [3]:
df = getNewZillowData()

In [4]:
df.shape

(77579, 67)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        223 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77579 non-null  float64
 3   heatingorsystemtypeid         49571 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      207 non-null    float64
 6   airconditioningtypeid         25007 non-null  float64
 7   parcelid                      77579 non-null  int64  
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77579 non-null  float64
 10  bedroomcnt                    77579 non-null  float64
 11  buildingqualitytypeid         49809 non-null  float64
 12  calculatedbathnbr             76963 non-null  float64
 13  d

In [6]:
df.describe() 

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,basementsqft,bathroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,223.000000,50.0,77579.000000,49571.000000,15.000000,207.000000,25007.000000,7.757900e+04,50.000000,77579.000000,...,17599.000000,172.0,7.746400e+04,7.757800e+04,77579.0,7.757700e+04,77574.000000,2900.000000,7.733200e+04,77579.000000
mean,6.040359,7.0,261.824476,3.921749,3.933333,7.386473,1.812013,1.300828e+07,679.720000,2.298496,...,1.434286,1.0,1.892811e+05,4.901506e+05,2016.0,3.011515e+05,5995.961788,14.088276,6.049667e+13,0.016803
std,0.556035,0.0,5.141596,3.594780,0.258199,2.728030,2.965768,3.519399e+06,689.703546,0.996732,...,0.544515,0.0,2.304107e+05,6.537979e+05,0.0,4.927249e+05,7628.859728,2.181281,1.533339e+12,0.170738
min,4.000000,7.0,31.000000,1.000000,3.000000,2.000000,1.000000,1.071186e+07,38.000000,0.000000,...,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,1.153820e+07,273.000000,2.000000,...,1.000000,1.0,8.417925e+04,2.068985e+05,2016.0,8.529300e+04,2712.630000,14.000000,6.037311e+13,-0.024309
50%,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,1.253054e+07,515.000000,2.000000,...,1.000000,1.0,1.364045e+05,3.588785e+05,2016.0,2.031880e+05,4448.265000,15.000000,6.037603e+13,0.006676
75%,6.000000,7.0,266.000000,7.000000,4.000000,7.000000,1.000000,1.421136e+07,796.500000,3.000000,...,2.000000,1.0,2.187340e+05,5.690000e+05,2016.0,3.667530e+05,6926.917500,15.000000,6.059042e+13,0.039291
max,13.000000,7.0,275.000000,24.000000,4.000000,21.000000,13.000000,1.676893e+08,3560.000000,18.000000,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


In [7]:
df.nunique().sort_values(ascending=False).head(30) #went back and dropped ID columns in getNewZillowData

parcelid                        77381
logerror                        77030
taxamount                       73825
latitude                        64038
longitude                       62461
landtaxvaluedollarcnt           51600
taxvaluedollarcnt               50949
structuretaxvaluedollarcnt      50430
rawcensustractandblock          39194
censustractandblock             39006
lotsizesquarefeet               18848
calculatedfinishedsquarefeet     4972
finishedsquarefeet12             4868
propertyzoningdesc               1907
finishedsquarefeet50             1807
finishedfloor1squarefeet         1787
finishedsquarefeet15             1724
garagetotalsqft                   839
yardbuildingsqft17                540
regionidneighborhood              480
regionidzip                       389
finishedsquarefeet6               350
transactiondate                   264
poolsizesum                       262
regionidcity                      175
yearbuilt                         136
propertycoun

##### Write a function that counts all the missing rows

In [8]:
def count_nulls(df,by_column=True):
    #intialize df
    df_nulls = pd.DataFrame()
    
    if by_column:
        #total rows
        num_rows = df.shape[0]
        #loop over each column
        for c in df.columns:
            #count # of nulls
            null_cnt = df[c].isna().sum()
            #calculate percent of nulls
            null_perc = null_cnt/num_rows
            #Populate DF
            df_nulls.loc[c,'num_nulls'] = null_cnt
            df_nulls.loc[c,'pct_nulls'] = null_perc
    else: 
        #total cols
        num_cols = df.shape[1]
        #loop over each row
        for i in df.index:
            #count # of nulls
            null_cnt = df.loc[i,:].isna().sum()
            #calculate percent of nulls
            null_perc = null_cnt/num_cols
            #Populate DF
            df_nulls.loc[i,'num_nulls'] = null_cnt
            df_nulls.loc[i,'pct_nulls'] = null_perc
    
    return df_nulls

In [9]:
count_nulls(df).sort_values(by='num_nulls',ascending=False)

,num_nulls,pct_nulls
buildingclassdesc,77564.0,0.999807
buildingclasstypeid,77564.0,0.999807
finishedsquarefeet13,77537.0,0.999459
storydesc,77529.0,0.999355
basementsqft,77529.0,0.999355
...,...,...
parcelid,0.0,0.000000
propertycountylandusecode,0.0,0.000000
bathroomcnt,0.0,0.000000
rawcensustractandblock,0.0,0.000000


In [10]:
#time intensive!
count_nulls(df,by_column=False).sort_values(by='num_nulls',ascending=False).head(5)

,num_nulls,pct_nulls
55652,48.0,0.716418
41670,48.0,0.716418
13457,48.0,0.716418
61287,47.0,0.701493
16644,47.0,0.701493


## Prepare

##### Remove properties that are unlikely to be single unit properties

Look at typeconstructiontypeid, lotsizesquarefeet, propertyzoningdesc, unitcnt, architecturalstyledesc, buildingclassdesc, propertylandusedesc

In [11]:
#unit count will likely take care of most of this
df.unitcnt.value_counts() #drop anything with more than 1 unit

1.0      47413
2.0       2031
4.0        726
3.0        529
45.0         1
42.0         1
6.0          1
237.0        1
Name: unitcnt, dtype: int64

In [12]:
df.typeconstructiondesc.value_counts() #ignore

Frame       220
Masonry       1
Metal         1
Concrete      1
Name: typeconstructiondesc, dtype: int64

In [13]:
df.propertyzoningdesc.value_counts() #would need to compare with their meaning in CA.  Ignore for now

LAR1         6766
LAR3         2450
LARS         1394
LBR1N        1225
LARD1.5      1132
             ... 
LCCM*           1
BFA16000*       1
LCPR*           1
LYR2            1
BFA15000*       1
Name: propertyzoningdesc, Length: 1907, dtype: int64

In [14]:
df.architecturalstyledesc.value_counts() #ignore, not enough information and no obvious issues

Contemporary     173
Conventional      19
Ranch/Rambler      7
Bungalow           5
Cape Cod           3
Name: architecturalstyledesc, dtype: int64

In [15]:
df.buildingclassdesc.value_counts() #very little info, mostly nulls anyway - Ignore

Buildings having wood or wood and steel frames                                                                                                                                                                                             14
Buildings having exterior walls built of a non-combustible material such as brick, concrete, block or poured concrete. Interior partitions and roof structures are built of combustible materials. Floor may be concrete or wood frame.     1
Name: buildingclassdesc, dtype: int64

In [16]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52441
Condominium                                   19342
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       1947
Quadruplex (4 Units, Any Combination)           729
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Residential General                              42
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [17]:
df.propertylandusetypeid.value_counts()

261.0    52441
266.0    19342
246.0     2021
269.0     1947
248.0      729
247.0      539
265.0      335
263.0       74
275.0       59
260.0       42
267.0       29
31.0        15
264.0        6
Name: propertylandusetypeid, dtype: int64

Per the [CA Builders website](https://cabuilderservices.com/planned-unit-development), PUDs can contain single family homes.  So it's possible new developments may not have the propertly land use type switched yet.

In [ ]:
#see if they are huge lots (big lots likely mean it's the entire development region, not individual units)
df[df.propertylandusetypeid == 269].lotsizesquarefeet.hist()

In [ ]:
#actually want to check against all possible values:
pd.read_sql('SELECT * FROM propertylandusetype',get_db_url('zillow'))

In [ ]:
#see if Duplexs are generally individual units or not
df[df.propertylandusetypeid == 246].unitcnt.value_counts()

Helpful resource for determining appropriate building type/zoning. [Zillow types of houses](https://www.zillow.com/resources/stay-informed/types-of-houses/)

**Action:** Leave in:
- Single Family (261)
- PUD (269)
- Mobile home (263)
- Townhouse (264)
- Condominium (266)
- Manufactured, etc (275)
- Residential general (260)
- Rural residence (262)
- Bungalow (273)
- Zero Lot Line (274)
- Inferred Single Family (279)
- Patio Home (276)
 
List to include: [261,269,263,264,266,275,260,262,273,274,279,276]

**Note:** Leaving mobile homes as they could have individual parcels.  Leaving PUDs, as they seem appropriate to include (inappropriate ones may be removed with outliers based of lot size.)  Not including duplexes/triplex/etc, even though they can be sold as individual units, it would be hard to know if they were multiple or not (in particular, whether or not we can rely on unitcnt to be accurate).  Excluding for this analysis.

##### Single unit filters:
- Only keep rows w/ unit counts of 1
- Only include these property land use type ids: [261,269,263,264,266,275,260,262,273,274,279,276] 


In [ ]:
df.shape

In [ ]:
#only keep rows with unit count of 1
df = df[df.unitcnt == 1]
#filter land use type
df = df[df.propertylandusetypeid.isin([261,269,263,264,266,275,260,262,273,274,279,276])]

In [ ]:
df.shape

##### Create a function to drop rows or columns based on the percent of values missing

In [ ]:
def handle_missing_values(df, prop_req_col=.75, prop_req_row=.75):
    """
    Checks the rows and columns for missing values.  Drops any rows or columns less than the specified percentage.
    
    Returnss: Dataframe
    Parameters: 
                df: Dataframe to analyze
      prop_req_col: Proportion (between 0 and 1) of values in column that must exist (non-null). Default: .75
      prop_req_row: Proportion (between 0 and 1) of values in a row that must exist (non-null). Default: .75
    """
    #Determine number of required non-nulls in columns (prop * num_rows)
    col_thresh = int(round(prop_req_col*df.shape[0],0))
    #drop columns w/o enough info
    df.dropna(axis=1,thresh=col_thresh,inplace=True)
    
    #NOW DO ROWS - note, this on already trimmed df
    #Determine number of required non-nulls in rows (prop * num_cols)
    row_thresh = int(round(prop_req_row*df.shape[1],0))
    #drop rows w/o enough info
    df.dropna(axis=0,thresh=row_thresh,inplace=True)
    
    return df

In [ ]:
handle_missing_values(df)
df.shape

### Imputing notes
- check out KNNImputer
  - Looks like fiting imputer can use subset of columns
  - Will take the mode, which means it won't work well for continuous columns. But, it may still be appropriate.  Finding the absolute closest neighbor and just using it's value.
- when would KNN be more appropriate


### Cleaning Notes
- Will want to drop ID columns